In [1]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, balanced_accuracy_score, roc_auc_score, average_precision_score, fbeta_score, matthews_corrcoef
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Lambda, Concatenate

In [3]:
# Custom function to build hidden combination (i.e. convolution) layers

def create_random_combination_layer(input_layer, combination_size, num_combinations, input_dim):
    outputs = []
    
    for _ in range(num_combinations):
        # First random feature selection
        indices_1 = np.random.choice(input_dim, combination_size, replace=False)
        indices_tensor_1 = tf.constant(indices_1, dtype=tf.int32)
        
        # First feature selection using Lambda layer
        slice_layer_1 = Lambda(
            lambda x: tf.gather(x, indices_tensor_1, axis=1),  # Gather selected features
            output_shape=(combination_size,)
        )(input_layer)
        
        # Second random feature selection (after the first random selection)
        indices_2 = np.random.choice(combination_size, combination_size, replace=False)
        indices_tensor_2 = tf.constant(indices_2, dtype=tf.int32)
        
        # Second feature selection using Lambda layer
        slice_layer_2 = Lambda(
            lambda x: tf.gather(x, indices_tensor_2, axis=1),  # Apply a second feature selection
            output_shape=(combination_size,)
        )(slice_layer_1)

        # Apply Dense layers on the final selected subset
        selected_features = Dense(16, activation='relu')(
            Dense(8, activation='relu')(
                Dense(4, activation='relu')(slice_layer_2)
            )
        )
        outputs.append(selected_features)
    
    # Concatenate the outputs from all the random feature combinations
    return Concatenate()(outputs)

In [ ]:
def run_model(train_data_path, test_data_path, is_string_labels = False, label_mapping = None):

    # Initialize the one-hot encoder for the target
    encoder = OneHotEncoder(sparse_output=False)

    # Load and Prepare Training Data
    train_data = pd.read_csv(train_data_path)
    train_data = train_data.sample(frac=1).reset_index(drop=True)  # Shuffle
    if (is_string_labels):
        train_data['label'] = train_data['label'].map(label_mapping)
    train_X = train_data.drop(columns=['label']).values
    train_y = train_data['label'].values
    train_y = encoder.fit_transform(train_y.reshape(-1, 1))

    # Perform a stratified split into train and validation sets (80% train, 20% validation)
    X_train, X_val, y_train, y_val = train_test_split(train_X, train_y, test_size=0.2, random_state=42, stratify=train_y)

    # Load and Prepare Test Data (this will not be used in training)
    test_data = pd.read_csv(test_data_path)
    test_data = test_data.sample(frac=1).reset_index(drop=True)  # Shuffle
    if (is_string_labels):
        test_data['label'] = test_data['label'].map(label_mapping)
    test_X = test_data.drop(columns=['label']).values
    test_y = test_data['label'].values
    test_y = encoder.transform(test_y.reshape(-1, 1))

    # Parameters for Random Feature Combination
    num_combinations = 20  # Number of random column combinations
    combination_size = 3   # Number of columns in each combination

    # Number of features and classes of the original dataset
    num_features = train_X.shape[1]
    num_classes = train_y.shape[1]

    # EarlyStopping Callback (optional, to avoid overfitting)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Number of runs for averaging results
    num_runs = 5

    # Initialize storage for metrics
    metrics_storage = defaultdict(list)

    # Train the Model with Validation Split N tines for more accurate metrics
    print("Verbose output only for first run...")
    verbose_run = 1
    for run in range(num_runs):
        
        # Model is defined separately in each run, since the random combination layers
        # must be randomly initialized each time. Otherwise, the "random" indices stay the same
        # throughout all runs
        input_layer = Input(shape=(X_train.shape[1],))  # Input shape from the training data
        feature_layer = create_random_combination_layer(input_layer, combination_size, num_combinations, X_train.shape[1])
        hidden_layer = Dense(128, activation='relu')(feature_layer)
        hidden_layer = Dropout(0.5)(hidden_layer)
        output_layer = Dense(test_y.shape[1], activation='softmax')(hidden_layer)
        model = Model(inputs=input_layer, outputs=output_layer)
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        print(f"Run {run + 1}/{num_runs} started...")
        history = model.fit(
            X_train, y_train, 
            epochs=1000, 
            batch_size=int(X_train.shape[0] * 0.05), 
            validation_data=(X_val, y_val),
            callbacks=[early_stopping],
            verbose=verbose_run
        )
        verbose_run = 0 # Suppress detailed output for multiple runs

        test_loss, test_acc = model.evaluate(test_X, test_y, verbose=0)
        y_pred = model.predict(test_X, verbose=0)
        y_pred_classes = y_pred.argmax(axis=1)
        y_true_classes = test_y.argmax(axis=1)

        # Compute metrics
        balanced_acc = balanced_accuracy_score(test_y, y_pred)
        roc_auc = roc_auc_score(test_y, y_pred, multi_class='ovr')
        pr_auc = average_precision_score(test_y, y_pred, average='weighted')
        f2 = fbeta_score(y_true_classes, y_pred_classes, beta=2, average='weighted')
        mcc = matthews_corrcoef(y_true_classes, y_pred_classes)

        # Store metrics
        metrics_storage['test_loss'].append(test_loss)
        metrics_storage['test_accuracy'].append(test_acc)
        metrics_storage['balanced_accuracy'].append(balanced_acc)
        metrics_storage['roc_auc'].append(roc_auc)
        metrics_storage['pr_auc'].append(pr_auc)
        metrics_storage['f2'].append(f2)
        metrics_storage['mcc'].append(mcc)

        # Store classification report metrics
        report = classification_report(y_true_classes, y_pred_classes, output_dict=True)
        for label, values in report.items():
            # Check if the value is a dictionary (e.g., 'precision', 'recall', 'f1-score')
            if isinstance(values, dict):
                for metric, value in values.items():
                    metrics_storage[f"{label}_{metric}"].append(value)
            else:
                # Handle scalar values (like 'accuracy')
                metrics_storage[label].append(values)

    # Average the metrics over all runs
    print("\nAggregated Metrics:")
    for metric, values in metrics_storage.items():
        avg_value = np.mean(values)
        print(f"{metric}: {avg_value:.4f}")
        

# Shuttle

In [5]:
run_model("shuttle_train.csv", "shuttle_test.csv", is_string_labels = False)

Verbose output only for first run...
Run 1/5 started...
Epoch 1/1000
20/20 ━━━━━━━━━━━━━━━━━━━━ 22s 58ms/step - accuracy: 0.4987 - loss: 1.6199 - val_accuracy: 0.8764 - val_loss: 0.6017
Epoch 2/1000
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.8723 - loss: 0.5206 - val_accuracy: 0.9471 - val_loss: 0.2632
Epoch 3/1000
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9299 - loss: 0.2774 - val_accuracy: 0.9648 - val_loss: 0.1876
Epoch 4/1000
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9624 - loss: 0.2042 - val_accuracy: 0.9670 - val_loss: 0.1483
Epoch 5/1000
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9667 - loss: 0.1481 - val_accuracy: 0.9664 - val_loss: 0.1266
Epoch 6/1000
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9695 - loss: 0.1273 - val_accuracy: 0.9672 - val_loss: 0.1102
Epoch 7/1000
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9701 - loss: 0.1174 - val_accuracy: 0.9675 - val_loss: 0.1000
Epoch 8/1000
20/20 ━━━━━━━━━━━━━━━━━━━━ 

# Covertype

In [6]:
run_model("covtype_train.csv", "covtype_test.csv", is_string_labels = False)

Verbose output only for first run...
Run 1/5 started...
Epoch 1/1000
21/21 ━━━━━━━━━━━━━━━━━━━━ 21s 176ms/step - accuracy: 0.3703 - loss: 1.9021 - val_accuracy: 0.3646 - val_loss: 1.6514
Epoch 2/1000
21/21 ━━━━━━━━━━━━━━━━━━━━ 3s 134ms/step - accuracy: 0.3975 - loss: 1.5202 - val_accuracy: 0.4973 - val_loss: 1.2262
Epoch 3/1000
21/21 ━━━━━━━━━━━━━━━━━━━━ 3s 135ms/step - accuracy: 0.4672 - loss: 1.2548 - val_accuracy: 0.4949 - val_loss: 1.1907
Epoch 4/1000
21/21 ━━━━━━━━━━━━━━━━━━━━ 3s 136ms/step - accuracy: 0.4741 - loss: 1.2232 - val_accuracy: 0.4979 - val_loss: 1.1849
Epoch 5/1000
21/21 ━━━━━━━━━━━━━━━━━━━━ 3s 144ms/step - accuracy: 0.4696 - loss: 1.2116 - val_accuracy: 0.4977 - val_loss: 1.1827
Epoch 6/1000
21/21 ━━━━━━━━━━━━━━━━━━━━ 3s 137ms/step - accuracy: 0.4809 - loss: 1.2090 - val_accuracy: 0.4977 - val_loss: 1.1822
Epoch 7/1000
21/21 ━━━━━━━━━━━━━━━━━━━━ 3s 134ms/step - accuracy: 0.4771 - loss: 1.2053 - val_accuracy: 0.4977 - val_loss: 1.1812
Epoch 8/1000
21/21 ━━━━━━━━━━━━━━

# KDD

In [7]:
labels_map = {
    'normal.': 0, 'satan.': 1, 'ipsweep.': 2, 'portsweep.': 3, 'nmap.': 4,
    'back.': 5, 'warezclient.': 6, 'teardrop.': 7, 'pod.': 8, 'guess_passwd.': 9,
    'buffer_overflow.': 10, 'land.': 11, 'warezmaster.': 12, 'imap.': 13, 'rootkit.': 14,
    'loadmodule.': 15, 'multihop.': 16, 'ftp_write.': 17, 'phf.': 18, 'perl.': 19, 'spy.': 20
}

run_model("kdd_train.csv", "kdd_test.csv", is_string_labels = True, label_mapping = labels_map)

Verbose output only for first run...
Run 1/5 started...
Epoch 1/1000
21/21 ━━━━━━━━━━━━━━━━━━━━ 24s 292ms/step - accuracy: 0.7939 - loss: 2.9304 - val_accuracy: 0.9551 - val_loss: 2.1358
Epoch 2/1000
21/21 ━━━━━━━━━━━━━━━━━━━━ 6s 259ms/step - accuracy: 0.9554 - loss: 1.5575 - val_accuracy: 0.9551 - val_loss: 0.3177
Epoch 3/1000
21/21 ━━━━━━━━━━━━━━━━━━━━ 5s 253ms/step - accuracy: 0.9556 - loss: 0.3429 - val_accuracy: 0.9551 - val_loss: 0.3270
Epoch 4/1000
21/21 ━━━━━━━━━━━━━━━━━━━━ 5s 253ms/step - accuracy: 0.9553 - loss: 0.3149 - val_accuracy: 0.9551 - val_loss: 0.2696
Epoch 5/1000
21/21 ━━━━━━━━━━━━━━━━━━━━ 5s 254ms/step - accuracy: 0.9553 - loss: 0.2895 - val_accuracy: 0.9551 - val_loss: 0.2675
Epoch 6/1000
21/21 ━━━━━━━━━━━━━━━━━━━━ 5s 256ms/step - accuracy: 0.9552 - loss: 0.2832 - val_accuracy: 0.9551 - val_loss: 0.2659
Epoch 7/1000
21/21 ━━━━━━━━━━━━━━━━━━━━ 5s 252ms/step - accuracy: 0.9548 - loss: 0.2825 - val_accuracy: 0.9551 - val_loss: 0.2636
Epoch 8/1000
21/21 ━━━━━━━━━━━━━━

# Darknet

In [8]:
labels_map = {
    'Normal': 0, 'Darknet_Audio-Streaming': 1, 'Darknet_Chat': 2, 'Darknet_File-Transfer': 3, 'Darknet_VOIP': 4,
    'Darknet_Video-Streaming': 5, 'Darknet_Email': 6, 'Darknet_Browsing': 7, 'Darknet_P2P': 8
}

run_model("darknet_train.csv", "darknet_test.csv", is_string_labels = True, label_mapping = labels_map)

Verbose output only for first run...
Run 1/5 started...
Epoch 1/1000
21/21 ━━━━━━━━━━━━━━━━━━━━ 18s 73ms/step - accuracy: 0.6664 - loss: 1.9174 - val_accuracy: 0.8467 - val_loss: 0.9577
Epoch 2/1000
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.8459 - loss: 0.8549 - val_accuracy: 0.8467 - val_loss: 0.6859
Epoch 3/1000
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.8459 - loss: 0.6944 - val_accuracy: 0.8465 - val_loss: 0.6330
Epoch 4/1000
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.8457 - loss: 0.6614 - val_accuracy: 0.8467 - val_loss: 0.6251
Epoch 5/1000
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.8463 - loss: 0.6502 - val_accuracy: 0.8467 - val_loss: 0.6186
Epoch 6/1000
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.8465 - loss: 0.6455 - val_accuracy: 0.8467 - val_loss: 0.6158
Epoch 7/1000
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.8451 - loss: 0.6417 - val_accuracy: 0.8467 - val_loss: 0.6116
Epoch 8/1000
21/21 ━━━━━━━━━━━━━━━━━━━━ 